## Post-Processing Notebook

Providing a use-case example for analysing and fitting the predictions of a previously ran TLCD-LSTM model.

In [ ]:
# General imports and widgets
import os
import sys
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 15
warnings.simplefilter("ignore")

import ipywidgets as widgets

In [ ]:
project_dir = os.path.join('/','Users','mario','git-repos','deepARTransit')  # change to yours

In [ ]:
# Radial aperture to help excluding the main star and compute background flux
def radial_aperture(array, cent, radius=3):
    aperture = np.empty_like(array, dtype=bool)
    if len(array.shape) == 2:
        aperture[:,:] = False
        for i in np.arange(array.shape[-2]):
            for j in np.arange(array.shape[-1]) :
                if (i + 0.5 - cent[0]) ** 2 + (j + 0.5  -cent[1]) ** 2 <= radius ** 2:
                    aperture[i,j] = True
    elif len(array.shape) == 3:
        aperture[:,:,:] = False
        for t in range(array.shape[0]):
            for i in np.arange(array.shape[-2]):
                for j in np.arange(array.shape[-1]) :
                    if (i + 0.5 - cent[0]) ** 2 + (j + 0.5  -cent[1]) ** 2 <= radius ** 2:
                        aperture[t, i,j] = True
    return aperture 

In [ ]:
mask = ~radial_aperture(np.ones(shape=(32,32)), (16,16), 16)

In [ ]:
experiment_widget = widgets.Dropdown(
    options=sorted(list({r.split('experiments')[1][1:] for r,d,f in 
                         os.walk('./experiments') if 'output' in d and 'summary' in d})),
    description='\t\t',
    disabled=False
)

### Selecting Experiment Folder

In [ ]:
experiment_widget

In [ ]:
from deepartransit.utils.config import get_config_file, process_config
from deepartransit.utils.data_handling import data_generator
from deepartransit.utils.transit import get_transit_model

experiment_folder = experiment_widget.value
experiment_dir = os.path.join('experiments', experiment_folder)
config_file = get_config_file(experiment_dir)

config = process_config(config_file)
data = data_generator.DataGenerator(config)
config = data.update_config()

t1, t2, t3 = config['pretrans_length'], config['trans_length'], config['postrans_length']
n_obs = data.Z.shape[0]

In [ ]:
time_arrays = np.arange(len(data.Z))
bts = np.zeros_like(data.Z)

In [ ]:
# Here I import time array corresponding to my data., ignore this cell or replace with your imports
from spitzerlc.data_handling import combine_fits_files
# For HD189733B
channel = 4
aorkey_list = ['22807296', '22807552', '22807808', '24537856', '27603712', '27773440']
parent_dir = '~/git-repos/spitzerLC/spitzerlc/data/agol_hd189733b/'

#WASP 121
channel = 2
aorkey_list = ['62160640']
parent_dir = '/Users/mario/data/IRAC/wasp121/'

from spitzerlc.observation import Observation
from spitzerlc.data_handling import load_data

time_arrays = np.zeros(data.Z.shape[:2])

background_mean = []
bts = []
for i,aorkey in enumerate(aorkey_list):
    time_array, flux, header = combine_fits_files(aorkey, channel, parent_dir = parent_dir)
    obs = Observation(aorkey, channel, header, time_array, flux )
    time_arrays[i] = obs.time_array
    
    #background_mean.append(np.nanmedian(obs.flux[200:500,mask], axis=1).mean()*25)
    bts.append(np.nanmedian(obs.flux[:,mask], axis=1) * 25)
    
    #print('background estimate', background_mean[-1])
    plt.plot(bts[-1])
    plt.show()
    
    obs.select_subregion(radius=2)
    plt.plot(obs.raw_light_curve().flux)
    plt.show()   
    plt.plot(obs.raw_light_curve().flux - bts[-1])
    plt.show()
time_arrays = time_arrays - 2454000

### Plotting input data

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20

fig, ax = plt.subplots(3, max(n_obs,2), figsize=(25,12), sharey='row', sharex='col')
Z_orig = data.scaler_Z.inverse_transform(data.Z)
for obs in range(n_obs):
    ax[0, obs].scatter(time_arrays[obs], data.Z[obs,:,0], 
                       s=3, label='raw light curve')
    ax[1, obs].plot(time_arrays[obs], data.X[obs,:,0], label='centroid X position', color='darkblue', alpha=0.5)
    ax[1, obs].plot(time_arrays[obs], data.X[obs,:,1], label='centroid Y position', color='darkgreen', alpha=0.5)
    ax[2, obs].plot(time_arrays[obs], data.X[obs,:,2], label='centroid X position', color='darkblue', alpha=0.5)
    ax[2, obs].plot(time_arrays[obs], data.X[obs,:,3], label='centroid Y position', color='darkgreen', alpha=0.5)

plt.tight_layout()
plt.legend()

fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.xlabel("Time [BMJD - 2454000 days]", fontsize=25, labelpad=30)
plt.subplots_adjust(bottom=0.8)


fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.ylabel("Normalized time-series", fontsize=25, labelpad=10)
#plt.subplots_adjust(bottom=0.8)

if 'plots' not in os.listdir(experiment_dir):
    os.mkdir(os.path.join(experiment_dir, 'plots'))
fig.savefig(os.path.join(experiment_dir, 'plots','data.png'))


# Loading results

In [ ]:
# retrieve evaluation steps
loc_dict = {}
pars_dict = {}
scale_dict = {}
for fn in [s for s in os.listdir(os.path.join(project_dir, config.output_dir)) if (s[:3] == 'loc')]:
    step = fn.split('_')[-1].split('.')[0]
    try:
        loc_dict[int(step)] = np.load(os.path.join(project_dir, config.output_dir, fn)).swapaxes(0,1)#.mean(axis=0)
        scale_dict[int(step)] = np.load(os.path.join(project_dir, config.output_dir, 'scales_array_{}.npy'.format(step))).swapaxes(0,1)#.mean(axis=0)
    except:
        print(fn)
        break
step_list = sorted([int(k) for k in loc_dict if k!='array'])
print(step_list)

In [ ]:
# Selection of optimal step
step = step_list[-1] # Last
step = 150

In [ ]:
loc_array = loc_dict[step]

In [ ]:
def to_real_time(t, true_ta, wrong_ta):
    return true_ta[0] + t * (true_ta[-1] - true_ta[0]) / (wrong_ta[-1] - wrong_ta[0])

time_arrays[0].shape, data.time_array[obs].shape

In [ ]:
fig, ax = plt.subplots(1, max(n_obs,2), figsize=(25,5), sharey='row')
for obs in range(n_obs):
    ax[obs].scatter(time_arrays[obs], data.Z[obs,:,0], s=3)
    ax[obs].plot(time_arrays[obs], loc_array[obs,:,0], label='prediction', color='red')
    ax[obs].ticklabel_format(axis='x',style='plain', useOffset=False)

    m, M = data.Z[obs,:,0].min(), data.Z[obs,:,0].max()
    ax[obs].vlines(to_real_time(t1-1, time_arrays[obs], data.time_array), 
                   m, M, 'black', linewidth=3, linestyles='dashed')
    ax[obs].vlines(to_real_time(t1+t2 + 1, time_arrays[obs], data.time_array), 
                   m, M, 'black', linewidth=3, linestyles='dashed')    
    
#plt.tight_layout()
ax[-1].legend()
ax[0].set_ylabel('Normalized Flux', fontsize=25)

ax1 = fig.add_subplot(111, frameon=False)
        
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.xlabel("Time [BMJD - 2454000 days]", fontsize=25, labelpad=-30)
#ax2 = plt.gca()
ax1.xaxis.set_label_coords(0.5, -0.13)    

# Fitting


### Linear Transit

In [ ]:
from deepartransit.utils.transit import LinearTransit
fig, ax = plt.subplots(2, max(n_obs,2), figsize=(25,9), sharey='row', sharex='col')

transit_component = np.zeros(shape=data.Z.shape)

ltransits = []
for obs in range(n_obs):
    
    transit_component[obs] = ((data.scaler_Z.inverse_transform(data.Z)[obs,:,0]-bts[obs])
                         / (data.scaler_Z.inverse_transform(loc_array)[obs,:,0]-bts[obs]))[:,np.newaxis]

    ax[0, obs].scatter(time_arrays[obs], data.Z[obs,:,0], s=3)
    ax[0, obs].plot(time_arrays[obs], loc_array[obs,:,0], label='prediction', color='red')
    ax[1, obs].scatter(time_arrays[obs], transit_component[obs,:,0], s=5)
    
    ltransit = LinearTransit(data.time_array)
    ltransit.fit(transit_component[obs,:,0], time_axis=0)
    ax[1, obs].plot(time_arrays[obs], ltransit.flux, label='prediction', color='red')
    print('{:.2f}'.format(ltransit.delta*100))
    mid_time = time_arrays[obs, 0] + ltransit.t_c * (time_arrays[obs, -1] - time_arrays[obs, 0]) / (data.time_array[-1] - data.time_array[0])
    ax[1, obs].vlines(mid_time, 0.995*(min(transit_component[obs])), 1, color='grey')
    
ax[0, obs].set_ylabel('Normalized Flux', fontsize=25)
ax[1, obs].set_ylabel('Star-Normalized Flux', fontsize=25)

ax1 = fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.xlabel("Time [BMJD - 2454000 days]", fontsize=25, labelpad=-30)
ax1.xaxis.set_label_coords(0.5, -0.13)    

if 'plots' not in os.listdir(experiment_dir):
    os.mkdir(os.path.join(experiment_dir, 'plots'))
plt.savefig(os.path.join(experiment_dir, 'plots','linear_fit.png'))

In [ ]:
import pandas as pd

y = transit_component[0,1:,0]
y_series = pd.Series(y)
mstd_width = 5
err = np.append(y_series[:5].std(), y_series.rolling(mstd_width,1).std().values[1:])

In [ ]:
err.shape

In [ ]:
plt.figure(figsize=(15,5))
plt.errorbar(time_arrays[obs,1:], y, err)

In [ ]:
np.save('wasp121_detrended_lc.npy', np.stack([time_arrays[obs,1:], y, err]).T)

In [ ]:
os.listdir('.')

In [ ]:
from pylightcurve import find_oec_parameters
(planet, logg, effective_temperature, metallicity, rp_over_rs, fp_over_fs,
 period, sma_over_rs, eccentricity, inclination, periastron, mid_time) = find_oec_parameters('hd189733b')


In [ ]:
(planet, logg, effective_temperature, metallicity, rp_over_rs, fp_over_fs,
 period, sma_over_rs, eccentricity, inclination, periastron, mid_time)

In [ ]:
inclination, sma_over_rs, period

In [ ]:
import time

from pylightcurve import TransitAndPolyFitting
import pandas as pd

fitting_list = []
ma_width = 50
mstd_width = 4

i = 0
fitting_folder = 'results_fitting'
if os.path.exists(os.path.join(experiment_dir, fitting_folder)):
    gmtime = time.gmtime(os.path.getmtime(os.path.join(experiment_dir, fitting_folder)))
    t = time.strftime('%y-%m-%d_%H-%M-%S', gmtime)
    os.rename(os.path.join(experiment_dir, fitting_folder), 
              os.path.join(experiment_dir, f'{fitting_folder}_{t}'))
os.mkdir(os.path.join(experiment_dir, fitting_folder))

for obs in range(n_obs):
    # estimate mid_time from linear fit
    mid_time = time_arrays[obs, 0] + ltransit.t_c * (time_arrays[obs, -1] - time_arrays[obs, 0]) / (data.time_array[-1] - data.time_array[0])
    print(mid_time)
    #x = (data.scaler_Z.inverse_transform(data.Z)[obs,:,0])
    #y = (x / pd.Series(x).rolling(ma_width,1).mean())
    y = pd.Series(transit_component[obs,:,0])
    err = np.append(y[:5].std(), y.rolling(mstd_width,1).std().values[1:])
    print(err.mean())
    transitFittingObject = TransitAndPolyFitting(data=[[time_arrays[obs], 
                                                        transit_component[obs,:,0],
                                                        err
                                                ]],
                                                method='linear',
                                                limb_darkening_coefficients=[0.1],
                                                rp_over_rs=rp_over_rs,
                                                period=period,
                                                sma_over_rs=sma_over_rs,
                                                eccentricity=eccentricity,
                                                inclination=inclination,
                                                periastron=periastron,
                                                mid_time = mid_time,
                                                iterations= 200_000,
                                                walkers= 100,
                                                burn= 100_000,
                                                precision=3,
                                                exp_time=0.,
                                                time_factor=1,
                                                fit_first_order=False,
                                                fit_second_order=False,
                                                fit_rp_over_rs=[0.01, 0.2],
                                                fit_period=False,
                                                fit_sma_over_rs = [sma_over_rs*0.5, sma_over_rs*2],
                                                fit_eccentricity = False,
                                                fit_inclination = [inclination*0.8, 90],
                                                fit_periastron = False,
                                                fit_mid_time = [mid_time - period /50, mid_time + period/50],
                                                fit_ld=[[0.0,0.5]],
                                                counter=True,
                                                counter_window=False)
    transitFittingObject.fit_ld = True
    transitFittingObject.run_mcmc()
    fitting_list.append(transitFittingObject.results)
    
    # Just printing
    rp_over_rs_new = transitFittingObject.results['parameters']['rp']
    print('Rp/Rs = {:.5f} -{:.6f}/+{:.6f}'.format(rp_over_rs_new['value'], 
                                                  rp_over_rs_new['m_error'], 
                                                  rp_over_rs_new['p_error']))
    print('(Rp/Rs)**2 = {:.5f} -{:.6f}/+{:.6f}'.format(rp_over_rs_new['value']**2, 
                                                       np.abs(2 * rp_over_rs_new['m_error'] * rp_over_rs_new['value']), 
                                                       np.abs(2 * rp_over_rs_new['p_error'] * rp_over_rs_new['value'])))
    
    # Just saving

    transitFittingObject.save_all(os.path.join(experiment_dir, fitting_folder, f'simulation_data_base_{obs}.pickle'))
    transitFittingObject.save_results(os.path.join(experiment_dir,fitting_folder, f'simulation_resultsobs_{obs}.txt'))
    transitFittingObject.plot_corner(os.path.join(experiment_dir,fitting_folder, f'simulation_correlations_{obs}.pdf'))
    transitFittingObject.plot_traces(os.path.join(experiment_dir,fitting_folder, f'simulation_traces_{obs}.pdf'))

In [ ]:
# Reload saved results
import pickle
fitting_folder = 'results_fitting'
with open(os.path.join(experiment_dir, fitting_folder, f'simulation_data_base_{obs}.pickle'), 'rb') as pick_file:
    transitFittingObject = pickle.load(pick_file)

In [ ]:
fig, ax = plt.subplots(3, max(n_obs,2), figsize=(25,14), sharey='row') #, sharex='col')
rol_win = 50
for obs in range(n_obs):
    ax[0,obs].scatter(time_arrays[obs], transit_component[obs,:,0], s=3)
    ax[0,obs].plot(time_arrays[obs], fitting_list[obs]['output_series']['model'], color='red', label='mcmc fit')
    
    res = fitting_list[obs]['output_series']['residuals']
    if obs==1:
        res = res.clip(-0.005, 0.005)
    ax[1,obs].scatter(time_arrays[obs], res, s=3)

    ma_res = pd.Series(res).rolling(rol_win, min_periods=1).mean().values
    mstd_res = pd.Series(res).rolling(rol_win, min_periods=1).std().values
    ax[1,obs].plot(time_arrays[obs], ma_res, color='red')
    ax[1,obs].fill_between(time_arrays[obs], ma_res - mstd_res, ma_res+mstd_res, alpha=0.5)

    ax[0,obs].ticklabel_format(axis='x',style='plain', useOffset=False)
    ax[1,obs].ticklabel_format(axis='x',style='plain', useOffset=False)
    ax[0,obs].tick_params(labeltop=True, labelbottom=False, top=True, bottom=False)
    ax[1,obs].tick_params(labeltop=False, labelbottom=False, top=False, bottom=False)
    ax[2, obs].bar(range(data.Z.shape[1]),fitting_list[obs]['detrended_statistics']['res_autocorr'] )

ax[0, 0].set_ylabel('Normalized Flux', fontsize=25)
ax[1, 0].set_ylabel('Residuals', fontsize=25)
ax[2, 0].set_ylabel('ACF', fontsize=25)
ax[2, 0].set_ylim(-0.25,0.25)
    
ax1 = fig.add_subplot(111, frameon=False)
#ax2 = ax1.twinx()
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.xlabel("Timesteps", fontsize=25, labelpad=-30)
#ax2 = plt.gca()
ax1.xaxis.set_label_coords(0.5, -0.05)     
    
    
#ax2 = fig.add_subplot(111, frameon=False)
ax2 = ax1.twiny()
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.margins(x = 1)
plt.xlabel("Time [BMJD - 2454000 days]", fontsize=25, labelpad=-30)
#ax2 = plt.gca()
ax2.xaxis.set_label_coords(0.5, 1.07) 
#ax.set_xlabel('xlabel', ha='left', va = 'top', )
#plt.subplots_adjust(bottom=0.8)


if 'plots' not in os.listdir(experiment_dir):
    os.mkdir(os.path.join(experiment_dir, 'plots'))
if 'mcmc_fit.png' in os.listdir(os.path.join(experiment_dir, 'plots')):
    ct = time.strftime('%y-%m-%d_%H-%M-%S',  
                       time.gmtime(os.path.getmtime(os.path.join(experiment_dir, 'plots','mcmc_fit.png'))))
    os.rename(os.path.join(experiment_dir, 'plots', 'mcmc_fit.png'),
              os.path.join(experiment_dir, 'plots', 'mcmc_fit_{}.png'.format(ct))
             )
plt.savefig(os.path.join(experiment_dir, 'plots','mcmc_fit.png'))

## Multi-observations: Plotting and printing

In [ ]:
# Defining weighted avg function, which computes both weighted average and std of this weighted average
def weighted_avg(x, sigma):
    x = np.array(x)
    sigma = np.array(sigma)
    try:
        if sigma.shape[1] == 2:
            sigma = sigma.mean(1)
    
    except IndexError:
        pass
    print(x.shape,sigma.shape)
    assert x.shape == sigma.shape
    
    weights = 1 / (np.array(sigma)**2)
    weighted_mean_delta = np.sum((x * weights )) / weights.sum()
    std_weighted_mean_delta = np.sqrt(1 / weights.sum())

    return weighted_mean_delta, std_weighted_mean_delta

In [ ]:
plt.figure(figsize=(15,5))

deltas = np.array([fitting_list[obs]['parameters']['rp']['value']**2 for obs in range(n_obs)])
deltas_errors = np.array([[fitting_list[obs]['parameters']['rp']['m_error']* rp_over_rs_new['value'] * 2,
        fitting_list[obs]['parameters']['rp']['p_error']* rp_over_rs_new['value'] * 2] 
       for obs in range(n_obs)])


mu, sigmu = weighted_avg(deltas, deltas_errors)
plt.errorbar(range(1, n_obs+1), 
             deltas, 
             yerr=deltas_errors.T,
             markersize=5, fmt='o',)
plt.hlines(mu, 1,6 , color= 'black')
plt.hlines(mu-sigmu, 1,6, linestyle='dashed', color= 'black')
plt.hlines(mu+sigmu, 1,6, linestyle='dashed', color= 'black')



In [ ]:
par_names_pylc = ['rp', 'ldc1', 'a','i']
par_names = ['$R_p/R_s$', '$u$', '$a/R_s$','$i$']
weighted_mu_dict = {}
f,ax = plt.subplots(len(par_names)//2,len(par_names)//2, figsize=(15, 8), sharex='col')
for i, parameter in enumerate(par_names_pylc):
    print(parameter)
    #param = fitting_list[obs]['parameters'][]
    
    par_values = [fitting_list[obs]['parameters'][parameter]['value'] for obs in range(n_obs)]
    par_errors = np.array([[fitting_list[obs]['parameters'][parameter]['m_error'],
                            fitting_list[obs]['parameters'][parameter]['p_error']] for obs in range(n_obs)])
    ix = i//2
    iy = (i % 2)
    print(ix,iy)
    ax[ix,iy].errorbar(range(1, n_obs+1), par_values, yerr=par_errors.T, markersize=5, fmt='o')
    
    mu, sigmu = weighted_avg(par_values, par_errors)
    ax[ix,iy].hlines(mu, 1,6 , color= 'black', 
                     label="$\overline{{{}}}={:.4g}$".format(par_names[i][1:-1], mu))
    ax[ix,iy].hlines(mu-sigmu, 1,6, linestyle='dashed', color= 'black')
    ax[ix,iy].hlines(mu+sigmu, 1,6, linestyle='dashed', color= 'black')
    ax[ix,iy].legend()
    ax[ix,iy].set_ylabel(par_names[i], fontsize=20)
    
    weighted_mu_dict[parameter] = mu
    
ax[-1, 0].set_xlabel('Observation number', fontsize=20)
ax[-1, 1].set_xlabel('Observation number', fontsize=20)

plt.show()

# Latex Print

In [ ]:
latex_string = ""
prec_val = [6,5,4,3,3]
for obs in range(6):
    latex_val_list = []
    latex_err_list = []
    for parameter in ['mt', 'rp', 'i', 'a', 'ldc1']:
        latex_val_list.append(fitting_list[obs]['parameters'][parameter]['value'])
        latex_err_list.append(max(fitting_list[obs]['parameters'][parameter]['m_error'], fitting_list[obs]['parameters'][parameter]['p_error']))
        if parameter == "mt":
            #latex_val_list[-1] -= 2454000
            pass
    latex_string += '\t'*2
    latex_string += " & ".join(["${0:.{2}f}\pm{{{1:.{3}f}}}$".format(latex_val_list[i], 
                                                                     latex_err_list[i],
                                                                     prec_val[i],
                                                                     prec_val[i]
                                                                    ) for i in range(len(latex_val_list))])
    latex_string += ' \\\\\n'
print(latex_string)

In [ ]:
# Just testing Latex rendering
from IPython.display import Latex
Latex(latex_string)

## Plot with Agol

In [ ]:
agol_deltas = np.array([2.4022, 2.4253, 2.4333, 2.4224, 2.3984, 2.3965])
agol_deltas_unc = np.array([47,63,51,49,62,74])/10_000
agol_deltas.shape, agol_deltas_unc.shape, deltas.shape, deltas_errors.shape

In [ ]:
# Scatter
agol_deltas.std(), np.array(deltas).std()

In [ ]:
deltas, agol_deltas

In [ ]:
wm, ws = weighted_avg(deltas*100, deltas_errors*100)
wma,wsa = weighted_avg(agol_deltas, agol_deltas_unc)

wm, wma, (wma - wm), (wma-wm)/ws

In [ ]:
plt.figure(figsize=(15,5))
p = plt.errorbar(np.arange(6)+0.02, 100*np.array(deltas), yerr=100*deltas_errors.T, 
             fmt='o', label='This paper', markersize = 8)
p2 = plt.errorbar(np.arange(6)-0.02, agol_deltas, yerr=agol_deltas_unc,  markersize = 8,
             fmt='o', label='Agol et al.')


plt.hlines(weighted_avg(100*deltas, 100*deltas_errors)[0], 0,5 , color= p[0].get_color())
plt.hlines(weighted_avg(100*deltas, 100*deltas_errors)[0] + weighted_avg(100*deltas, 100*deltas_errors)[1], 
           0,5, linestyle='dashed', color= p[0].get_color())
plt.hlines(weighted_avg(100*deltas, 100*deltas_errors)[0] - weighted_avg(100*deltas, 100*deltas_errors)[1], 
           0,5, linestyle='dashed', color= p[0].get_color())


plt.hlines(weighted_avg(agol_deltas, agol_deltas_unc)[0], 0,5 , color= p2[0].get_color())
plt.hlines(weighted_avg(agol_deltas, agol_deltas_unc)[0] + weighted_avg(deltas, agol_deltas_unc)[1], 
           0,5, linestyle='dashed', color= p2[0].get_color())
plt.hlines(weighted_avg(agol_deltas, agol_deltas_unc)[0] - weighted_avg(deltas, agol_deltas_unc)[1], 
           0,5, linestyle='dashed', color= p2[0].get_color())


plt.legend()
plt.xlabel('Observation Number', fontsize=25)

plt.ylabel('$(R_p/R_*)^2$  $[\%]$', fontsize=25)
pass

## Second run with $i$, $u$ and $a/R_S$ fixed


In [ ]:
fitting_list_2 = []
ma_width = 50
mstd_width = 5
for obs in range(n_obs):
    # estimate mid_time from linear fit
    mid_time = time_arrays[obs, 0] + ltransit.t_c * (time_arrays[obs, -1] - time_arrays[obs, 0]) / (data.time_array[-1] - data.time_array[0])
    
    y = pd.Series(transit_component[obs,:,0])
    err = np.append(y[:5].std(), y.rolling(mstd_width,1).std().values[1:])
    transitFittingObject = TransitAndPolyFitting(data=[[time_arrays[obs], 
                                                        transit_component[obs,:,0],
                                                        err
                                                ]],
                                                method='linear',
                                                limb_darkening_coefficients=[weighted_mu_dict['ldc1']], ##########
                                                rp_over_rs=weighted_mu_dict['rp'],
                                                period=period,
                                                sma_over_rs=weighted_mu_dict['a'], #########
                                                eccentricity=eccentricity,
                                                inclination=weighted_mu_dict['i'], ########
                                                periastron=periastron,
                                                mid_time = mid_time,
                                                iterations= 50_000,
                                                walkers= 100,
                                                burn= 25_000,
                                                precision=3,
                                                exp_time=0.,
                                                time_factor=1,
                                                fit_first_order=False,
                                                fit_second_order=False,
                                                fit_rp_over_rs=[0.14, 0.17],
                                                fit_period=False,
                                                fit_sma_over_rs = False,
                                                fit_eccentricity = False,
                                                fit_inclination = False,
                                                fit_periastron = False,
                                                fit_mid_time = [mid_time - period /50, mid_time + period/50],
                                                fit_ld=False,
                                                counter=True,
                                                counter_window=False)
    transitFittingObject.fit_ld = True
    transitFittingObject.run_mcmc()
    fitting_list_2.append(transitFittingObject.results)
    
    # Just printing
    rp_over_rs_new = transitFittingObject.results['parameters']['rp']
    print('Rp/Rs = {:.5f} -{:.6f}/+{:.6f}'.format(rp_over_rs_new['value'], 
                                                  rp_over_rs_new['m_error'], 
                                                  rp_over_rs_new['p_error']))
    print('(Rp/Rs)**2 = {:.5f} -{:.6f}/+{:.6f}'.format(rp_over_rs_new['value']**2, 
                                                       np.abs(2 * rp_over_rs_new['m_error'] * rp_over_rs_new['value']), 
                                                       np.abs(2 * rp_over_rs_new['p_error'] * rp_over_rs_new['value'])))
    
    # Just saving
    transitFittingObject.save_all(f'fit_results/simulation_data_base_{obs}_2.pickle')
    transitFittingObject.save_results(f'fit_results/simulation_resultsobs_{obs}_2.txt')
    transitFittingObject.plot_corner(f'fit_results/simulation_correlations_{obs}_2.pdf')
    transitFittingObject.plot_traces(f'fit_results/simulation_traces_{obs}_2.pdf')

In [ ]:
plt.figure(figsize=(15,5))


deltas = np.array([fitting_list_2[obs]['parameters']['rp']['value']**2 for obs in range(n_obs)])
deltas_errors = np.array([[fitting_list_2[obs]['parameters']['rp']['m_error']* rp_over_rs_new['value'] * 2,
        fitting_list[obs]['parameters']['rp']['p_error']* rp_over_rs_new['value'] * 2] 
       for obs in range(n_obs)])


mu, sigmu = weighted_avg(deltas, deltas_errors)
plt.errorbar(range(1, n_obs+1), 
             deltas, 
             yerr=deltas_errors.T,
             markersize=5, fmt='-o',)
plt.hlines(mu, 1,6 , color= p[0].get_color())
plt.hlines(mu-sigmu, 1,6, linestyle='dashed', color= 'black')
plt.hlines(mu+sigmu, 1,6, linestyle='dashed', color= 'black')
